In [ ]:
# Load pacakges ----
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from IPython.display import HTML
from IPython.display import Markdown as md
from IPython.core.display import HTML as Center
from config import marker, qof_measure_marker, start_date, end_date, demographics, codelist_path
from IPython.display import Image, display
import base64

from utilities import *
%matplotlib inline

# QOF - Blood Pressure
## Indicator: BP002 (Version: 46.0)
## Timeframe: 2019-09-01 and 2022-03-31

# Content

<ul id="docNav">
<li> <a href="#intro">Introduction</a>
<li> <a href="#methods">Methods</a>
    <ul>
    <li> <a href="#business_rules">Business rules</a>
        <ul>
            <li> <a href="#business_rules_bp002">Indicator BP002</a>
                <ul>
                    <li> <a href="#bp002_denominator">Denominator</a>
                    <li> <a href="#bp002_numerator">Numerator</a>
                </ul>
                <li> <a href="#bp002_codelists">Codelists</a>
        </ul>
</ul>
<li> <a href="#results">Results</a>
    <ul>
        <li> <a href="#results_bp002">Indicator BP002</a>
             <ul>
                 <li> <a href="#results_bp002_total">Total</a>
                 <li> <a href="#results_bp002_var_gp">Variation across GP practices</a>
                 <li> <a href="#results_bp002_by_sex">Breakdown by sex</a>
                 <li> <a href="#results_bp002_by_age_band">Breakdown by age band</a>
                 <li> <a href="#results_bp002_by_region">Breakdown by region</a>
                 <li> <a href="#results_bp002_by_imd">Breakdown by IMD quintiles</a>
                 <li> <a href="#results_bp002_by_ethnicity">Breakdown by ethnicity</a>
                 <li> <a href="#results_bp002_by_learning_disability">Breakdown by learning disability</a>
                 <li> <a href="#results_bp002_by_care_home_status">Breakdown by NHS care home status</a>
            </ul>
    </ul>
<li> <a href="#discussion">General discussion</a>
<li> <a href="#references">References</a>

</ul>

<a id="intro"></a>
# Introduction

> *The Quality and Outcomes Framework (QOF) is a voluntary annual reward and incentive programme for all GP practices in England* ([NHSD, 2021a](https://qof.digital.nhs.uk/)). 
> *The objective of the Quality and Outcomes Framework (QOF) is to improve the quality of care patients are given by rewarding practices for the quality of care they provide to their patients, based on a number of indicators across a range of key areas of clinical care and public health.*
> *QOF participation by GP practices is very high, with 96.7% of practices participating in the 2020/21 publication* ([NHSD, 2021b](https://digital.nhs.uk/data-and-information/publications/statistical/quality-and-outcomes-framework-achievement-prevalence-and-exceptions-data/2020-21)).

OpenSAFELY is a secure analytics platform for electronic patient records built by our group on behalf of NHS England to deliver urgent academic and operational research during the pandemic (e.g., [Curtis et al., 2021](https://bjgp.org/content/72/714/e63); [Williamson et al., 2020](https://www.nature.com/articles/s41586-020-2521-4)). 
Analyses can currently run across all patients' full pseudonymised primary care records, with patient-level linkage to various sources of secondary care data. 
The OpenSAFELY-TPP is a Trusted Research Environment (TRE) containing the routine clinical data of 23.4m people, approximately 40% of England’s population.

Currently QOF reports are published at the end of the financial year, and consist of prevalence rates and indicator achievement rates for clinical areas by NHS geographic breakdowns from region to practice.
We set out to replicate the indicator logic for a variety of QOF clinical indicators in OpenSAFELY-TPP and describe trends monthly over time, before and during the pandemic.
We also describe how the indicators vary between key clinical, regional and demographic subgroups.

High blood pressure is one of the leading risk factors for several diseases (e.g., cardiovascular disease, stroke) worldwide. 
Research suggests that delays in the management of high blood pressure are associated with worse clinical outcomes, for example acute cardiovascular events, or death ([Xu et al., 2015](https://www.bmj.com/content/350/bmj.h158)).
This report describes the results of implementing the blood pressure clinical indicator, the percentage of patients aged 45 or over who have a record of blood pressure in the preceding 5 years in the OpenSAFELY-TPP TRE. 


<a id="methods"></a>
# Methods

Using OpenSAFELY-TPP, covering 40% of England's population, we have assessed QOF Blood Pressure targets (Indicator: BP002) between 2019-09-01 and 2022-03-31.
The codelist used can be found here at [OpenSAFELY Codelists](https://codelists.opensafely.org/).

The **study population** for each indicator is defined by the denominator rules shown above.
For each month within the study period, we have calculated the percentage of registered patients above 45 years that had their blood pressure recorded in the 5 year period leading up to the month of reporting.
All analytical code and output is available for inspection at the [OpenSAFELY GitHub repository](https://github.com/opensafely/blood-pressure-qof).

<a id="business_rules"></a>

## Business rules

The denominator and numerator rules for BP002 (Version: 46.0) are outlined below.
Prevalence data is not available for the blood pressure indicator. 
More details about the rules can be found [here](https://digital.nhs.uk/data-and-information/data-collections-and-data-sets/data-collections/quality-and-outcomes-framework-qof/quality-and-outcome-framework-qof-business-rules/qof-business-rules-v46.0-2021-2022-baseline-release).
Dashboards presenting the annual targets of all QOF indicators published by NHSD are available [here](https://digital.nhs.uk/data-and-information/data-tools-and-services/data-services/general-practice-data-hub/quality-outcomes-framework-qof).

<a id="business_rules_bp002"></a>

### Indicator BP002

<a id="bp002_denominator"></a>

#### Denominator

| Rule number | Rule | Action if true | Action if false | Rule description or comments |
|:---|:---|:---|:---|:---| 
| 1 | If `PAT_AGE` < 45 | Reject | Next rule | Reject patients from the specified population who are aged less than 45 years old. |
| 2 | If `BP_DAT` > (`PPED` – 5 years) | Select | Next rule | Select patients passed to this rule who had their blood pressure recorded in the 5 year period leading up to and including the payment period end date. |
| 3 | If `BPDEC_DAT` > (`PPED` – 5 years)  | Reject | Next rule | Reject patients passed to this rule chose not to have their blood pressure recorded in the 5 year period leading up to and including the payment period end date. |
| 4 | If `REG_DAT` > (`PPED` – 3 months) | Reject | Select | Reject patients passed to this rule who registered with the GP practice in the 3 month period leading up to and including the payment period end date. |

<a id="bp002_numerator"></a>

#### Numerator

| Rule number | Rule | Action if true | Action if false | Rule description or comments |
|:---|:---|:---|:---|:---| 
| 1 | If `BP_DAT` > (`PPED` – 5 years) | Select | Reject | Select patients from the denominator who had their blood pressure recorded in the 5 year period leading up to and including the payment period end date. Reject the remaining patients. |

<a id="codelists"></a>

## Codelists

### NHSD Primary Care Domain Refsets 

Following the QOF business rules for blood pressure (v47) the clinical code clusters below were used.
Further information about the business rules and codelists can be found [here](https://content.digital.nhs.uk/qofesextractspecs) or using the hyperlinks of the cluster names in the table below.

| Cluster name | Description | SNOMED CT |
|:---|:---|:---|
| [BP_COD](https://www.opencodelists.org/codelist/nhsd-primary-care-domain-refsets/bp_cod/20200812/) | Blood pressure (BP) recording codes | ^999012731000230108 |
| [BPDEC_COD](https://www.opencodelists.org/codelist/nhsd-primary-care-domain-refsets/bpdec_cod/20200812/) | Codes indicating the patient has chosen not to have blood pressure procedure | ^999012611000230106 |

Futher studies looking at blood pressure may also need to investigate home blood pressure management (see [HOMEBP_COD](https://www.opencodelists.org/codelist/nhsd-primary-care-domain-refsets/homebp_cod/20200812/) and [HOMEBPDEC_COD](https://www.opencodelists.org/codelist/nhsd-primary-care-domain-refsets/homebpdec_cod/20211221/)).

### Codelists used for breakdowns

The following codelists are used for breakdowns of indicators.

| Codelist name | Description | Coding system |
|:---|:---|:---|
| [Ethnicity](https://www.opencodelists.org/codelist/opensafely/ethnicity/2020-04-27) | A list of ethnicity codes in use in UK general practice including aggregate grouping at two levels (6 and 16). | CTV3 |
| [NHS England Care Homes residential status](https://www.opencodelists.org/codelist/opensafely/nhs-england-care-homes-residential-status/3712ef13) | This codelists supports analysis of records of people who may reside in a nursing or care home. | SNOMED CT |
| [Learning disabilities](https://www.opencodelists.org/codelist/opensafely/learning-disabilities/2020-07-06) | The following codes are used to indicate learning disability. | CTV3 |




<a id="results"></a>
# Results

<a id="results_bp002"></a>
## Indicator BP002

The overall yearly achievement percentage in QOF 2019-22 was **97.38%** for indicator **BP002**.
Dashboards presenting the annual targets of all QOF indicators published by NHSD are available [here](https://digital.nhs.uk/data-and-information/data-tools-and-services/data-services/general-practice-data-hub/quality-outcomes-framework-qof).


In [ ]:
codelist = pd.read_csv(f'../{codelist_path}')

image_paths = {d: f'../output/plot_{d}.png' for d in demographics}
image_paths['total'] = '../output/plot_total.png'

<a id="results_bp002_total"></a>

### Percentage of patients above 45 years that had their blood pressure recorded in the last 5 years

In [ ]:
Image(filename=image_paths['total'])

<a id="results_bp002_var_gp"></a>
### Total Number by GP Practice

In [ ]:
practice_table = pd.read_csv('../output/rate_table_practice.csv', parse_dates=['date']).sort_values(by='date')
percentage_practices = get_percentage_practices(practice_table)
md(f"Percentage of practices with a recording of a code within the [BP_COD](https://www.opencodelists.org/codelist/nhsd-primary-care-domain-refsets/bp_cod/20200812/) codelist during the study period: {percentage_practices}%")

### Percentage of patients above 45 years that had their blood pressure recorded in the last 5 years across GP practices
<!-- Description of indicator BP002 -->

In [ ]:
Image(filename='../output/decile_chart.png')

<a id="results_bp002_by_sex"></a>
### Breakdown by sex

In [ ]:
Image(filename=image_paths[demographics[0]])

<a id="results_bp002_by_age_band"></a>
### Breakdown by age band

In [ ]:
Image(filename=image_paths[demographics[1]])

<a id="results_bp002_by_region"></a>

### Breakdown by region

In [ ]:
Image(filename=image_paths[demographics[2]])

<a id="results_bp002_by_imd"></a>

### Breakdown by IMD quintiles

In [ ]:
Image(filename=image_paths[demographics[3]])

<a id="results_bp002_by_ethnicity"></a>

### Breakdown by ethnicity

In [ ]:
Image(filename=image_paths[demographics[4]])

<a id="results_bp002_by_learning_disability"></a>

### Breakdown by learning disability


In [ ]:
display(Image(filename=image_paths[demographics[5]]))

<a id="results_bp002_by_care_home_status"></a>

### Breakdown by NHS care home status

In [ ]:
Image(filename=image_paths[demographics[6]]) 

<a id="discussion"></a>
# General discussion

- Compared to the total yearly achievement percentage of 97.38% for indicator BP002 in QOF 2019-22, our results indicate slightly lower values overall for the time period from 2019-09-01 to 2022-03-31.
- The variation in achievement percentage across all GP practices was consistent throughout the study period, with a slight increase in variation over time (see deciles chart).
- Observed differences in percentage achievement across the subgroups were mainly as expected. The results indicate that higher risk groups (e.g., patients aged 80+) were prioritised and did experience a lower decrease in achievement compared to lower risk groups (e.g., patients aged between 45 and 49).


<a id="references"></a>
# References 

1. Curtis HJ, MacKenna B, Croker R, Inglesby P, Walker AJ, Morley J, et al. OpenSAFELY NHS Service Restoration Observatory 1: describing trends and variation in primary care clinical activity for 23.3 million patients in England during the first wave of COVID-19. Br J Gen Pract [Internet]. 2021 Sep 20 [cited 2021 Oct 5]; Available from: https://bjgp.org/content/early/2021/09/24/BJGP.2021.0380
1. NHS Digital. QOF 2020-21 results. 2021a. https://qof.digital.nhs.uk/
1. NHS Digital. Quality and Outcomes Framework, 2020-21 Official Statistics. 2021b. https://digital.nhs.uk/data-and-information/publications/statistical/quality-and-outcomes-framework-achievement-prevalence-and-exceptions-data/2020-21
1. Williamson EJ, Walker AJ, Bhaskaran K, Bacon S, Bates C, Morton CE, et al. Factors associated with COVID-19-related death using OpenSAFELY. Nature. 2020 Aug;584(7821):430–6. https://www.nature.com/articles/s41586-020-2521-4
1. Xu W, Goldberg SI, Shubina M, Turchin A. Optimal systolic blood pressure target, time to intensification, and time to follow-up in treatment of hypertension: population based retrospective cohort study. BMJ. 2015 Feb 5;350:h158. https://www.bmj.com/content/350/bmj.h158

